In [57]:
import numpy as np
from matplotlib import pyplot as plt
from math import floor
import argparse
import os
from skimage.filters.thresholding import threshold_li, threshold_yen
import multiprocessing as mp
from scipy import ndimage
from tifffile import imread, imwrite
from skimage.morphology import disk, skeletonize
import cv2
import multiprocessing as mp
from itertools import repeat
import tifffile as ti
import skimage

from scipy import ndimage, fftpack, stats
from matplotlib import pyplot as plt
from math import floor, ceil

from skimage import restoration
from skimage.util import img_as_uint
from skimage.filters import unsharp_mask, median

import matplotlib
matplotlib.rcParams['figure.figsize'] = [10, 10]

from skimage.morphology import remove_small_holes, remove_small_objects

In [58]:
from pylabel import importer

dataset = importer.ImportYoloV5("/mnt/external.data/TowbinLab/spsalmon/test_coco_database/labels/labels_yolo/", path_to_images="/mnt/external.data/TowbinLab/spsalmon/test_coco_database/labels/images/", img_ext="tiff", cat_names=['zizou', 'head', 'tail'])
dataset_dataframe = dataset.df

list_images = np.unique(dataset_dataframe['img_filename'])
output_folder = "/mnt/external.data/TowbinLab/spsalmon/test_coco_database/labels/yolo_masks/"

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

Importing YOLO files...: 100%|██████████| 85/85 [00:03<00:00, 28.30it/s]


In [60]:
for image in list_images:
    image_data = dataset_dataframe[dataset.df['img_filename'] == image]
    image_dim = (image_data['img_width'].iloc[0], image_data['img_height'].iloc[0])
    output_mask = np.zeros(image_dim, dtype="uint8")
    df = image_data.reset_index()  # make sure indexes pair with number of rows

    for index, row in df.iterrows():
        bbox_xmin = int(row['ann_bbox_xmin'])
        bbox_ymin = int(row['ann_bbox_ymin'])
        bbox_width = int(row['ann_bbox_width'])
        bbox_height = int(row['ann_bbox_height'])


        if row['cat_name'] == "tail":
            output_mask[bbox_ymin:(bbox_ymin + bbox_height), bbox_xmin:(bbox_xmin + bbox_width)] = 7
        if row['cat_name'] == "head":
            output_mask[bbox_ymin:(bbox_ymin + bbox_height), bbox_xmin:(bbox_xmin + bbox_width)] = 3

    ti.imwrite(os.path.join(output_folder, image), output_mask, compression="zlib")